<a href="https://colab.research.google.com/github/lodimk2/0404_python_workshop/blob/main/Python_USB_Workshop_ANSWERKEY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# USB Python Workshop
# Unsupervised Machine Learning Principles
# scRNAseq Basic Analysis Principles
# K Means Clustering Implementation
# Presented by: Musaddiq Lodi

# K Means Tutorial Based on: https://medium.com/@avijit.bhattacharjee1996/implementing-k-means-clustering-from-scratch-in-python-a277c23563ac
# Also Based on https://towardsdatascience.com/create-your-own-k-means-clustering-algorithm-in-python-d7d4c9077670

# GOAL: Implement K Means Clustering algorithm on scRNAsequencing data

In [3]:
# Let's start by loading in the scRNAseq data we will be working with.
# For ease of loading we will be using the scanpy package.

!pip install scanpy
import scanpy as sc



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.1/122.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 3.2 MB/s eta 0:00:00
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8026 sha256=f4bfb37cbc2ccd22379eac3c5f4022e352038cd9c205814190a93c6ffddaa92b
  Stored in directory: /root/.cache/pip/wheels/6a/aa/b9/eb5d4031476ec10802795b97ccf937b9bd998d68a9b268765a
Successfully built session-info


In [4]:
import pandas as pd
# Now, we will read in the input file of the count matrix as a pandas dataframe
github_url = "https://raw.githubusercontent.com/lodimk2/0404_python_workshop/main/workshop_scdata.csv"

# Read in the file as a pandas dataframe. We know that the first column should be row names, hence the index_col parameter being set.
data = pd.read_csv(github_url, index_col = 0)


In [5]:
# Lets inspect our data

# View top 5 rows and columns
print(data.iloc[:5, :5])


# Check the number of rows and columns in the data
print("Number of rows in dataframe:", data.shape[0])
print("Number of columns in dataframe:", data.shape[1])

         AGGGAACGA-GATTGCGA  TGAAGGATTCA-GGTTTCTC  ACTCCGCAT-GTTAACCA  \
Foxo6                     0                     0                   0   
Trip4                     0                     0                   2   
Slc15a3                   0                     0                   0   
Papss1                    0                     0                   1   
Gm13031                   0                     0                   0   

         GAGCGGTA-CTTAGGTA  TGATCGACACC-ACTAGGAT  
Foxo6                    0                     0  
Trip4                    0                     1  
Slc15a3                  0                     0  
Papss1                   0                     1  
Gm13031                  0                     0  
Number of rows in dataframe: 500
Number of columns in dataframe: 100


In [6]:
# Next, we will create a scanpy object from the dataframe.
# Using a scanpy object will allow for basic scRNAseq tasks, such as:
# 1) Normalizing the data
# 2) Selecting highly variable features
# 3) Clustering and visualizing

# Let's walk through some basic preprocessing and quality control steps common to scRNAsequencing data.

# Common convention is to call a scanpy object "adata"

adata = sc.AnnData(data)
print(adata)


AnnData object with n_obs × n_vars = 500 × 100


In [7]:
# PAUSE!! Let's take a second to check our data and make sure that it makes sense. What should the observations be and what should the variables be?


# Let's remake the scanpy object with the correct dimensions. Scanpy objects are created from matrices where the genes are the columns, and the cells are the rows.

# How can we remake the object?

adata = sc.AnnData(data.T)
print(adata)

# Always remember to INSPECT your data!


AnnData object with n_obs × n_vars = 100 × 500


In [8]:
# Now that the scanpy object has the correct dimensions, we can continue with some preprocessing steps.

# Optionally, you can also filter out bad genes
sc.pp.filter_genes(adata, min_cells=3)  # Remove genes detected in less than 3 cells

# Perform additional preprocessing steps as needed, such as normalization, scaling, etc.
sc.pp.normalize_total(adata, target_sum=1e4)  # Normalize total counts per cell
sc.pp.log1p(adata)  # Log-transform the data



In [9]:
# In order to have an accurate comparison to perform the downstream clustering, we need the normalized and scaled data.

# Access the normalized and scaled data
log_transformed_matrix = adata.X

print(adata.X)

# Inspect it

print(log_transformed_matrix[:5, :5])

[[0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        4.2480025 3.569047  ... 0.        0.        3.569047 ]
 ...
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]]
[[0.        0.        0.        0.        0.       ]
 [0.        0.        0.        0.        4.091399 ]
 [0.        4.2480025 3.569047  0.        5.3370404]
 [0.        0.        0.        0.        4.1327586]
 [0.        3.7574956 3.7574956 3.7574956 4.4389033]]


In [10]:
# Now we will work on the K-Means clustering algorithm. The K-Means clustering algorithm may be broadly summarized into 5 steps:
# 1) Choose the number of clusters, k, that you want to create.
# 2) Initialize k cluster centroids randomly.
# 3) Assign each data point to the nearest centroid, creating k clusters.
# 4) Recalculate the centroids as the mean of all data points in each cluster.
# 5) Repeat steps 3 and 4 until convergence (centroids no longer change significantly) or for a specified number of iterations.

In [11]:
import numpy as np

class KMeans:
    def __init__(self, n_clusters, max_iters=100):
        self.n_clusters = n_clusters
        self.max_iters = max_iters

    def fit(self, X):
        # Initialize centroids randomly
        self.centroids = self._initialize_centroids(X)

        for _ in range(self.max_iters):
            # Assign each data point to the nearest centroid
            labels = self._assign_labels(X)

            # Update centroids
            new_centroids = self._update_centroids(X, labels)

            # Check for convergence
            if np.array_equal(self.centroids, new_centroids):
                break

            self.centroids = new_centroids

    def _initialize_centroids(self, X):
        # Randomly select initial centroids from data points
        indices = np.random.choice(X.shape[0], self.n_clusters, replace=False)
        return X[indices]

    def _assign_labels(self, X):
        labels = []
        for point in X:
            # Compute distances from the point to centroids
            distances = [np.linalg.norm(point - centroid) for centroid in self.centroids]
            # Assign label based on the nearest centroid
            labels.append(np.argmin(distances))
        return np.array(labels)

    def _update_centroids(self, X, labels):
        new_centroids = []
        for i in range(self.n_clusters):
            # Extract data points assigned to the current cluster
            cluster_points = X[labels == i]
            # Compute the mean of data points to get the new centroid
            new_centroid = np.mean(cluster_points, axis=0)
            new_centroids.append(new_centroid)
        return np.array(new_centroids)


In [12]:
# Now, lets test the clusters that we made

# Create a K-Means instance with 10 clusters
kmeans = KMeans(n_clusters=10)

# Fit the model to the data
kmeans.fit(log_transformed_matrix)

# Get cluster assignments for each data point
labels = kmeans._assign_labels(log_transformed_matrix)

In [13]:
print(labels)

[1 4 4 2 6 0 3 1 0 0 4 2 2 9 4 4 6 1 0 0 5 3 1 2 1 4 8 3 1 9 0 0 3 4 1 1 4
 3 6 7 4 2 1 1 0 0 0 3 2 3 3 0 1 2 3 8 1 3 1 4 1 3 4 6 1 0 9 1 1 0 1 3 3 6
 0 0 0 0 7 3 0 6 0 0 0 4 8 0 6 1 0 1 1 6 0 1 0 0 3 0]


In [ ]:
# There we go! We have now performed clustering on our dataset, and obtained the predicted labels.
# There are several other steps we can go from here to obtain biologically relevant conclusions related to cell states.